# By-Band _g_-Point Reduction

In [ ]:
%load_ext autoreload
%autoreload 2

# Dependencies

`numpy` is installed in the Python environment at NERSC (`module load python`), but `xarray` is not, so the user must install the package on their own. `PIPPATH` is the assumed location. This notebook depends heavily on `xarray`.

In [ ]:
# standard libraries
import os, sys, shutil, glob, pickle
from multiprocessing import Pool

# "standard" conda install (NERSC already provides this without user 
# having to install it)
import numpy as np

# conda installs
from tqdm import tqdm

# directory in which libraries installed with conda are saved
PIPPATH = '{}/.local/'.format(os.path.expanduser('~')) + \
    'cori/3.7-anaconda-2019.10/lib/python3.7/site-packages'
PIPPATH = '/global/homes/e/emlawer/.local/cori/3.8-anaconda-2020.11/' + \
     'lib/python3.8/site-packages'
PIPPATH = '/global/homes/p/pernak18/.local/cori/3.9-anaconda-2021.11/lib/python3.9/site-packages'
PATHS = ['common', PIPPATH]
for path in PATHS: sys.path.append(path)

# needed at AER unless i update `pandas`
import warnings
#warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=Warning)

# user must do `pip install xarray` on cori (or other NERSC machines)
import xarray as xa

# local modules
import g_point_reduction as REDUX
from rrtmgp_cost_compute import flux_cost_compute as FCC

# Static Inputs

In [ ]:
# only do one domain or the other
DOLW = True
DOMAIN = 'LW' if DOLW else 'SW'
NBANDS = 16 if DOLW else 14

# does band-splitting need to be done, or are there existing files 
# that have divided up the full k-distribution?
BANDSPLIT = False

# Paths

In [ ]:
PROJECT = '/global/project/projectdirs/e3sm/pernak18/'
EXE = '{}/g-point-reduction/garand_atmos/rrtmgp_garand_atmos'.format(
    PROJECT)
REFDIR = '{}/reference_netCDF/g-point-reduce'.format(PROJECT)

# test (RRTMGP) and reference (LBL) flux netCDF files, full k-distributions, 
# and by-band Garand input file
fluxSuffix = 'flux-inputs-outputs-garandANDpreind.nc'
if DOLW:
    GARAND = '{}/multi_garand_template_single_band.nc'.format(REFDIR)
    KFULLNC = '{}/rrtmgp-data-lw-g256-2018-12-04.nc'.format(REFDIR)
    KFULLNC = '{}/rrtmgp-data-lw-g256-jen-xs.nc'.format(REFDIR)
    REFNC = '{}/lblrtm-lw-{}'.format(REFDIR, fluxSuffix)
    TESTNC = '{}/rrtmgp-lw-{}'.format(REFDIR, fluxSuffix)
    #TESTNC = 'rrtmgp-lw-flux-inputs-outputs-garand-all.nc'
else:
    GARAND = '{}/charts_multi_garand_template_single_band.nc'.format(REFDIR)
    KFULLNC = '{}/rrtmgp-data-sw-g224-2018-12-04.nc'.format(REFDIR)
    REFNC = '{}/charts-sw-{}'.format(REFDIR, fluxSuffix)
    TESTNC = '{}/rrtmgp-sw-{}'.format(REFDIR, fluxSuffix)
# endif LW

BANDSPLITDIR = 'band_k_dist'
FULLBANDFLUXDIR = 'full_band_flux'

for PATH in PATHS: FCC.pathCheck(PATH)

CWD = os.getcwd()

KPICKLE = '{}_k-dist.pickle'.format(DOMAIN)
pickleCost = '{}_cost-optimize.pickle'.format(DOMAIN)

# Band Splitting

Break up full _k_-distribution file into separate distributions for each band, then calculate the corresponding fluxes. This should only need to be run once.

After some clarifications from Robert (30-Nov-2020), I believe the plan of action is:

1. create Nbands k-distribution files
2. drive the Fortran executable Nbands times to produce Nbands flux results
3. the trial g-point combinations then loop over bands and the possible g-point combinations within each band, creating k-distribution and band-wise flux files for each possible combination
4. The Python code assembles broadband fluxes from the band-wise flux files in order to compute the cost functions

In [ ]:
if BANDSPLIT:
    print('Band splitting commenced')
    FCC.pathCheck(BANDSPLITDIR, mkdir=True)
    FCC.pathCheck(FULLBANDFLUXDIR, mkdir=True)
    kFiles, fullBandFluxes = [], []
    for iBand in tqdm(range(NBANDS)):
        # divide full k-distribution into subsets for each band
        kObj = REDUX.gCombine_kDist(KFULLNC, iBand, DOLW, 1, 
            fullBandKDir=BANDSPLITDIR, fullBandFluxDir=FULLBANDFLUXDIR, 
            profilesNC=GARAND)
        kFiles.append(kObj.kBandNC)
        kObj.kDistBand()

        # quick, non-parallelized flux calculations (because the 
        # executable is run in one directory)
        FCC.fluxCompute(kObj.kBandNC, kObj.profiles, kObj.exe, 
                           kObj.fullBandFluxDir, kObj.fluxBandNC)
        fullBandFluxes.append(kObj.fluxBandNC)
    # end band loop
    print('Band splitting completed')
else:
    kFiles = sorted(glob.glob('{}/coefficients_{}_band??.nc'.format(
        BANDSPLITDIR, DOMAIN)))
    fullBandFluxes = sorted(glob.glob('{}/flux_{}_band??.nc'.format(
        FULLBANDFLUXDIR, DOMAIN)))

    if len(kFiles) == 0 or len(fullBandFluxes) == 0:
        print('WARNING: set `BANDSPLIT` to `True` and run this cell again')
# endif BANDSPLIT


# Pressure Levels for Cost Function

Pressure levels [Pa] for the Garand atmospheres are printed to standard output with indices that can be used in the cost function:

In [ ]:
with xa.open_dataset(REFNC) as refDS:
    pLev = refDS['p_lev'].isel(record=0)
for iLev, pLev in enumerate(pLev.isel(col=0).values): print(iLev, pLev)

# _g_-Point Combining

Combine _g_-point reduced for bands with full-band fluxes from other bands, find optimal _g_-point combination for given iteration, proceed to next iteration.

First, find all _g_-point combinations for each band. Store the band object in a dictionary for use in flux computation. This cell only needs to be run once, and to save time in development, the dictionary is saved in a `pickle` file and can be loaded in the next cell.

In [ ]:
# this should be parallelized; also is part of preprocessing so we 
# shouldn't have to run it multiple times
kBandDict = {}
for iBand, kFile in tqdm(enumerate(kFiles)):
    #if iBand != 0: continue
    band = iBand + 1
    kObj = REDUX.gCombine_kDist(kFile, iBand, DOLW, 1, 
        fullBandKDir=BANDSPLITDIR, 
        fullBandFluxDir=FULLBANDFLUXDIR)
    kObj.gPointCombine()
    kBandDict['band{:02d}'.format(band)] = kObj

    print('Band {} complete'.format(band))
# end kFile loop

import pickle
with open(KPICKLE, 'wb') as fp: pickle.dump(kBandDict, fp)

Now compute fluxes in parallel for every _g_-point combination -- merging occurs in each band, and these combinations in a given band are used with broadband fluxes from other bands. These concatenations each have an associated `xarray` dataset assigned to it. Cost function components are then calculated based for each dataset, and the one that minimizes the error in the cost function will have its associated netCDF saved to disk.

Uncomment pickling block to restore dictionary from previous cell.

# Reduction and Optimization

Test and reference netCDF files have flux and heating rate arrays of dimension `record` x `col` x `lay`/`lev` and `band` if the array is broken down by band. `record` represents atmospheric specifications that can be used in [forcing scenarios](https://github.com/pernak18/g-point-reduction/wiki/LW-Forcing-Number-Convention#g-point-reduction-convention-).

Alternatively, the atmospheric specifications from any scenario can also be used. "Bare" parameters like `heating_rate` and `flux_net` will be treated as PD specifications, so the user will have to specify explicitly if they want the fluxes or heating rates from other scenarios by using the `flux_*_N` and `heating_rate_N` convention, where `N` is the scenario index as listed in the above list. The same convention applies to band fluxes and HRs. `N` = 0 will work just like `heating_rate` and `flux_net`.

Forcing for this exercise is defined as PI subtracted from scenario (2-6). The convention for these quantities is `*_forcing_N`, where `*` is the typical flux or heating rate (band or broadband) string, and `N` again is the forcing scenario (`N` of 2 would be forcing due to doubling methane).

First, let's define the cost function (component names, levels/layers, and weights):

In [ ]:
# pickling for developement purposes so this dictionary doesn't need 
# to be regenerated for every code change.
with open(KPICKLE, 'rb') as fp: kBandDict = pickle.load(fp)

# components used in cost function computation
# variable names in RRTMGP and LBL flux netCDF file, except for 
# forcing, which has to be specifed with "_forcing" appended to 
# the appropriate array. e.g., "flux_net_forcing" for net flux forcing
# netCDF arrays ('heating_rate', 'flux_net', 'band_flux_net', etc.)
# or forcing scenarios: convention is  ('flux_net_forcing_3') for 
#CFCOMPS = ['flux_dif_net', 'flux_dir_dn', 'heating_rate']
CFCOMPS = ['flux_net','band_flux_net','heating_rate','heating_rate_7',
           'flux_net_forcing_5','flux_net_forcing_6','flux_net_forcing_7',
           'flux_net_forcing_9','flux_net_forcing_10','flux_net_forcing_11',
           'flux_net_forcing_12','flux_net_forcing_13','flux_net_forcing_14',
           'flux_net_forcing_15','flux_net_forcing_16','flux_net_forcing_17',
           'flux_net_forcing_18']
# level indices for each component 
# (e.g., 0 for surface, 41 for Garand TOA)
# one dictionary key per component so each component
# can have its own set of level indices
CFLEVS = {}
CFLEVS['flux_net'] = [0, 26, 42]
CFLEVS['band_flux_net'] = [42]
CFLEVS['heating_rate'] = range(42)
CFLEVS['heating_rate_7'] = range(42)
CFLEVS['flux_net_forcing_5'] = [0, 26, 42]
CFLEVS['flux_net_forcing_6'] = [0, 26, 42]
CFLEVS['flux_net_forcing_7'] = [0, 26, 42]
CFLEVS['flux_net_forcing_9'] = [0, 26, 42]
CFLEVS['flux_net_forcing_10'] = [0, 26, 42]
CFLEVS['flux_net_forcing_11'] = [0, 26, 42]
CFLEVS['flux_net_forcing_12'] = [0, 26, 42]
CFLEVS['flux_net_forcing_13'] = [0, 26, 42]
CFLEVS['flux_net_forcing_14'] = [0, 26, 42]
CFLEVS['flux_net_forcing_15'] = [0, 26, 42]
CFLEVS['flux_net_forcing_16'] = [0, 26, 42]
CFLEVS['flux_net_forcing_17'] = [0, 26, 42]
CFLEVS['flux_net_forcing_18'] = [0, 26, 42]

# weights for each cost function component
CFWGT = [0.6, 0.04, 0.12, 0.12,
         0.01, 0.02, 0.04,
        0.005, 0.005, 0.005,
        0.005, 0.005, 0.005, 
        0.005, 0.005, 0.005,
        0.005]

# directory under which to store k-distribution files that optimize 
# the cost function for each iteration and diagnistics (if necessary)
CFDIR = 'fullCF_top-layer_redo_abs_parabola'
FCC.pathCheck(CFDIR, mkdir=True)

Now we starting combining _g_-points and calculating costs for all combinations, then select the trial that minimizes the cost for a given iteration. Rinse and repeat.

In [ ]:
### write diagnostic netCDFs with cost function components
DIAGNOSTICS = True

RESTORE = True

if RESTORE:
    assert os.path.exists(pickleCost), 'Cannot find {}'.format(pickleCost)
    print('Restoring {}'.format(pickleCost))
    with open(pickleCost, 'rb') as fp: coObj = pickle.load(fp)
else:
    # instantiate object for computing cost
    coObj = REDUX.gCombine_Cost(
        kBandDict, fullBandFluxes, REFNC, TESTNC, 1, 
        DOLW, profilesNC=GARAND, exeRRTMGP=EXE, 
        costFuncComp=CFCOMPS, costFuncLevs=CFLEVS, 
        costWeights=CFWGT, optDir='./{}'.format(CFDIR))
# endif RESTORE

# number of iterations for the optimization
NITER = 94

COSTCUT = 0.1

for i in range(coObj.iCombine, NITER+1):
    print('Iteration {}'.format(i))
    coObj.kMap()
    coObj.fluxComputePool()
    coObj.fluxCombine()
    if i == 1: coObj.costFuncComp(init=True)
    coObj.costFuncComp()
    coObj.findOptimal()
    if coObj.optimized: break
    if DIAGNOSTICS: coObj.costDiagnostics()

    # Start of special g-point combination branch
    dCostIter = np.abs(coObj.totalCost[coObj.iOpt]-coObj.winnerCost)
    if dCostIter > COSTCUT: break

    # coObj = REDUX.modCombine(coObj, i, coObj.optBand, 
    #     diagnostics=DIAGNOSTICS, 
    #     kDirIn=BANDSPLITDIR, fluxDirIn=FULLBANDFLUXDIR)

    coObj.setupNextIter()
    with open(pickleCost, 'wb') as fp: pickle.dump(coObj, fp)
    if i in [90, 93, 100, 114]:
        with open('LW_cost-optimize_iter{:03d}.pickle'.format(i), 'wb') as fp: pickle.dump(coObj, fp)
    coObj.calcOptFlux(
        fluxOutNC='optimized_{}_fluxes_iter{:03d}.nc'.format(DOMAIN, i))
# end iteration loop

KOUTNC = 'rrtmgp-data-{}-g-red.nc'.format(DOMAIN)

# TO DO: getting this error:
# `Using a DataArray object to construct a variable is ambiguous, please extract the data using the .data property.`
# but not sure how to fix -- downgrading xarray didn't work
# it's not vital, though, for now -- we have all the information to RESTORE then try building this 
# again when we think we have a fix, as long as the files are not deleted
# coObj.kDistOpt(KFULLNC, kOutNC=KOUTNC)

coObj.calcOptFlux(fluxOutNC='optimized_{}_fluxes.nc'.format(DOMAIN))

# Modified _g_-Point Combining

At this point, we have reached large enough delta-costs -- i.e., the cost difference with respect to the **previous** iteration -- such that we would like to implement the modified _g_-point combining [described](https://github.com/pernak18/g-point-reduction/wiki/Modified-g-Point-Combining) by Eli. [Issue 19](https://github.com/pernak18/g-point-reduction/issues/19) details the refinements on this modified combination approach, namely:

- parabolas are not necessary used in finding the zero crossing anymore; Eli found that the trends of scaling the weights and their associated delta-costs were approximately linear
- we need to focus on all remaining trials, not just the "winner" of a given iteration as we did in Karen's implementation of the modified combining (methods with `sgl` in their names)

The latter item is the stickler because we have to calculate fluxes and associated costs for all remaining trials over a few more "weight scales" that are the abscissa to go along with their delta-cost ordinates, which will be used in determining zero crossings and thus the scale weight that minimizes the delta-cost. Currently, we scale the _g_-point weights with `iniWgt` scaled with 0 ("init"), 1 ("plus"), and 2 ("2plus"), which is the naming convention used by Karen. We may revisit how many data points we use, given that a) the flux and cost computations are expensive (time), and b) delta-costs are linear with weight. 

First, we have to apply the modified combinations and generate their corresponding _k_-distribution file:

In [ ]:
# print(coObj.fluxInputsAll[0].keys())
# print(coObj.fluxInputsAll[-1]['bandID'])
# print(coObj.optBand, coObj.iOpt, coObj.optNC)
# print(coObj.fluxInputsAll[0]['fluxNC'])

# gPointCombineSglPair scales initial weights with either 
# a factor of 1 or 2
iniWgt = 0.05
scaleWeight = ['plus', '2plus']

kBand = coObj.distBands

kBandDict = {}
kBandDict['init'] = dict(kBand)

for sWgt in scaleWeight:
    print(sWgt)
    # replace existing kBandDict with modified objects
    kBandDict[sWgt] = {}
    for iBand, key in enumerate(kBand.keys()):
        band = iBand+1
        # print('{}-Modified k for band {:d}'.format(sWgt, band))
        kNC = kBand[key].kInNC
        kObjMod = REDUX.gCombine_kDist(kNC, iBand, DOLW, i, 
            fullBandKDir=BANDSPLITDIR, fullBandFluxDir=FULLBANDFLUXDIR)

        # how many combinations exist for band?
        with xa.open_dataset(kNC) as ds: nCombine = ds.dims['gpt']-1

        # g-point indices, and its combination "partner"
        gCombine = [[x, x+1] for x in range(nCombine)]

        # generate the k-dist netCDF for each combination in band
        for comb in gCombine:
            kObjMod.gPointCombineSglPair(sWgt, [comb], iniWgt)
        # end gCombine loop

        kBandDict[sWgt]['band{:02d}'.format(band)] = kObjMod
    # end kBand loop
# end scaleWeight loop

outPickle = KPICKLE.replace('k-dist', 'mod-k-dist')
with open(outPickle, 'wb') as fp: 
    pickle.dump(kBandDict, fp)

print('Wrote k-dist (modified reduction) objects to {}'.format(outPickle))
# TO DO: remove trial NC for scaleWeights

Now, we continue with those _k_-distributions in flux and cost computation, which is done exactly like we did with the original combinations. Note for diagnostics, we need to sync the modified cost optimization object (`coObjMod`) with original cost-optimization object (`coObj`) up to the current iteration where modification is first applied. Then we save the delta-costs for all trials in all weight scales to be used in the zero-crossing calculation.

In [ ]:
dCostMod = {}
dCostMod['init'] = coObj.totalCost - coObj.winnerCost

for sWgt in scaleWeight:
    coObjMod = REDUX.gCombine_Cost(
        kBandDict[sWgt], fullBandFluxes, REFNC, TESTNC, i, 
        DOLW, profilesNC=GARAND, exeRRTMGP=EXE, 
        costFuncComp=CFCOMPS, costFuncLevs=CFLEVS, 
        costWeights=CFWGT, optDir='./{}'.format(CFDIR))

    coObjMod.kMap()
    coObjMod.fluxComputePool()
    coObjMod.fluxCombine()

    # use coObj init costs so we don't have to do that calculation again
    coObjMod.optNC = coObj.optNC
    coObjMod.costComp0 = coObj.costComp0
    coObjMod.iOpt = coObj.iOpt

    # grab components up until current, modified iteration
    for comp, cost0 in coObj.cost0.items():
        coObjMod.cost0[comp] = cost0[:-1]
        coObjMod.dCost0[comp] = coObj.dCost0[comp][:-1]
        coObjMod.costComps[comp] = coObj.costComps[comp][:-1]
        coObjMod.dCostComps[comp] = coObj.dCostComps[comp][:-1]
        coObjMod.costComp0[comp] = coObj.costComp0[comp]
        coObjMod.dCostComps0[comp] = coObj.dCostComps0[comp]
    # end comp loop

    # proceed with subsequent cost optimization methods
    coObjMod.costFuncComp()
    coObjMod.costDiagnostics()
    dCostMod[sWgt] = coObjMod.totalCost - coObj.winnerCost
# end scaleWeight loop

pickleCost = pickleCost.replace('cost-optimize', 'mod-cost-optimize')
with open(pickleCost, 'wb') as fp: pickle.dump(coObjMod, fp)

In [ ]:
# fit line for zero-crossing estimation
abscissa = np.array([0, 1, 2])
ordinates = np.vstack([dCost for dCost in dCostMod.values()])
coeffs = np.polyfit(abscissa, ordinates, 1)

# for noCross, we keep whatever scale weight produced (positive) minimum
# and do not need to re-run flux/cost calcs
# new scale factors for weights will start as just the scale that 
# minimized dCost, then we fill in where a zero crossing was found
iMinDelta = np.argmin(ordinates, axis=0)
yMin = np.nanmin(ordinates, axis=0)
xMin = (abscissa[iMinDelta])
y0 = ordinates[0, :]

# was there a crossing at a given trial? minimum must be negative
cross = yMin <= 0
noCross = yMin > 0

newScales = np.array(iMinDelta).astype(float)
newScales[cross] = (xMin - xMin * yMin / (yMin - y0))[cross]
iNan = np.where(np.isnan(newScales))
y0[iNan] = ordinates[2, iNan]
newScales[iNan] = (xMin - xMin * yMin / (yMin - y0))[iNan]

Another round of _k_-distribution modifications to create the netCDFs with the zero-crossing scale weights that were found in the linear regression:

In [ ]:
import pathlib as PL

gCombineAll = []
for iBand, key in enumerate(kBand.keys()):
    band = iBand+1
    kNC = kBand[key].kInNC

    # how many combinations exist for band?
    with xa.open_dataset(kNC) as ds: nCombine = ds.dims['gpt']-1

    # g-point indices, and its combination "partner"
    gCombineAll += [[x, x+1] for x in range(nCombine)]
# end band loop

# restore modified cost-optimation object (plus/2plus scaling)
with open(pickleCost, 'rb') as fp: coObjMod = pickle.load(fp)

# loop over all trials with a zero crossing
iReprocess = np.where(cross)[0]
for iRep in tqdm(iReprocess):
    # grab k-distribution for each band BEFORE g-point combining for this iteration
    fluxInputs = coObjMod.fluxInputsAll[iRep]
    kNC = coObj.distBands['band{:02d}'.format(fluxInputs['bandID']+1)].kInNC

    g1, g2 = gCombineAll[iRep]

    # generate modified k-dist netCDF file with new scale weight
    kObjMod = REDUX.gCombine_kDist(kNC, fluxInputs['bandID'], 
        DOLW, i, fullBandKDir=BANDSPLITDIR, fullBandFluxDir=FULLBANDFLUXDIR)
    kObjMod.gPointCombineSglPair('regress', [[g1, g2]], newScales[iRep]*iniWgt)

    # replace flux computations i/o with modified files
    coObjMod.fluxInputsAll[iRep]['kNC'] = fluxInputs['kNC'].replace('2plus', 'regress')
    coObjMod.fluxInputsAll[iRep]['fluxNC'] = fluxInputs['fluxNC'].replace('2plus', 'regress')
    coObjMod.fluxInputsAll[iRep]['fluxDir'] = str(fluxInputs['fluxDir']).replace('2plus', 'regress')
# end reprocessing loop

In [ ]:
from copy import deepcopy as DCP
coObjRep = DCP(coObjMod)

# reprocessing should only be done over trials with a zero crossing
varsReduced = ['fluxInputsAll', 'trialNC', 'combinedNC', 'dCost', 'totalCost']
iZero = np.where(cross)[0]
for vr in varsReduced: setattr(coObjRep, vr, list(np.array(getattr(coObjRep, vr))[iZero]))

coObjRep.fluxComputePool()
coObjRep.fluxCombine()
coObjRep.costFuncComp()

In [ ]:
# grab components up until current, modified iteration
coObjRep.totalCost = []
coObjRep.dCost = []
coObjRep.winnerCost = float(coObj.winnerCost)
for comp, cost0 in coObj.cost0.items():
    coObjRep.cost0[comp] = cost0[:-1]
    coObjRep.dCost0[comp] = coObj.dCost0[comp][:-1]
    coObjRep.costComps[comp] = coObj.costComps[comp][:-1]
    coObjRep.dCostComps[comp] = coObj.dCostComps[comp][:-1]
    coObjRep.costComp0[comp] = coObj.costComp0[comp]
    coObjRep.dCostComps0[comp] = coObj.dCostComps0[comp]
# end comp loop

coObjRep.costFuncComp()

Now we need to consolidate the reprocessed costs with modified-but-not-reprocessed (i.e., no zero crossing) trials

In [ ]:
# coObjMod.costFuncComp()
with open(pickleCost, 'rb') as fp: coObjMod = pickle.load(fp)

coObjMod.winnerCost = float(coObj.winnerCost)
for iRep, iMod in enumerate(iReprocess): 

    coObjMod.totalCost[iMod] = float(coObjRep.totalCost[iRep])
    for comp, cost0 in coObjRep.cost0.items():
        coObjMod.costComps[comp][iMod] = coObjRep.costComps[comp][iRep]
        coObjMod.dCostComps[comp][iMod] = coObjRep.dCostComps[comp][iRep]
        coObjMod.costComp0[comp] = coObjRep.costComp0[comp]
        coObjMod.dCostComps0[comp] = coObjRep.dCostComps0[comp]
    # end comp loop
# end reprocess loop

coObjMod.findOptimal()
coObjMod.costDiagnostics()

Now that one iteration with modified combining is complete, we can proceed similarly to more iterations. This has the potentially to be **very** time consuming, even parallelized on a supercomputer (because of I/O? Python parallelization overhead? unsure why...), so from here on out, we'll try to use the _k_-distribution and cost optimization objects we've already taken the time to generate and only replace trials for the band that contained the winner in the previous iteration. While we're doing this only with modified combining, there is no reason this logic cannot be applied to iterations where the modified combining is not applied.

Eventually, all of the cells from modified combining will be placed into `g_point_reduction.py` and called in this notebook.

Spitballing how to do this:
- `kDistBand` retention, exploit `optBand` attribute again (we already do this in a number of places in `g_point_combination.py`
- `coObjMod` trial lists -- determine indices of winner band, `costFuncComp` on only newly modified trials in said band (should only be one batch), then insert results into appropriate `coObjMod` lists at appropriate indices